In [ ]:
import pandas as pd
import loader as load
# aak_ge = load.loadGEWithClinical()

In [ ]:
import processor as pr

tcma_cancer_type = ["COAD", "ESCA", "HNSC", "READ", "STAD"]

data = [load.loadTCMA("genus"), load.loadTCMA("phylum"), aak_ge, load.loadGEOverlappingTCMA("genus"), load.loadGEOverlappingTCMA("phylum")]
source_types = ["GENUS", "PHYLUM", "GE", "GENUS n GE", "PHYLUM n GE"]
file_names = ["TCMA_Genus", "TCMA_Phylum", "TCGA_GE(Aakash)", "TCMA_Genus_GE(Aakash)_Overlap", "TCMA_Phylum_GE(Aakash)_Overlap"]
reduction_type = "TSNE"
for i, b in enumerate(data):
    X, Y, titles = [], [], []
    b = load.attachTumorStatus(b)
    for c in tcma_cancer_type:
        x, y = pr.splitData(b, "tumor", project=c)
        if reduction_type == "PCA":
            X.append(pr.getPCA(x))
        else:
            X.append(pr.getTSNE(x))
        Y.append(y)
        titles.append(f"{c} {source_types[i]} {reduction_type}")
    pr.plotScatter(X, Y, titles, f"Visual/{reduction_type}/{file_names[i]}")


In [ ]:
target = "tumor"
files = ['aak_ge', 'tcma_gen', 'tcma_gen_aak_ge']
data = [pd.read_csv(fr"Data\Descriptor\Prediction_Tables\{target}\{x}_pred.csv", index_col=None) for x in files]
data[0].head(5)
data[0]["cancer"].nunique()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# fig, ax = plt.subplots()
fig = plt.figure(figsize = (18, 6))

metric = "f1-score"

print(data[0].p.unique())
for i, p in enumerate(data[0].p.unique()):
    ax = fig.add_subplot(1,3,(i+1)) 
    ax.set_title(f"{target} p: {p}", fontsize = 10)
    ax.set_ylabel(metric, fontsize = 15)
        

    for j, d in enumerate(data):
        d = d[[metric, "cancer", "p"]]
        d = d[d.p == p]
        line = d.plot(kind="line", x="cancer", y=metric, ax=ax, label=files[j])
filename = f"Visual/prediction/{target}_{metric}.png"
plt.savefig(filename, transparent=False, facecolor="white")
